In [ ]:
# Spatial index for geopandas sjoin_nearest
!apt-get -qq install -y libspatialindex-dev

# Geospatial + utils
!pip -q install osmnx==1.9.1 geopandas shapely rtree networkx requests tqdm pyproj pandas numpy folium

# Streamlit + ngrok
!pip -q install streamlit streamlit-folium pyngrok

Selecting previously unselected package libspatialindex6:amd64.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../libspatialindex6_1.9.3-2_amd64.deb ...
Unpacking libspatialindex6:amd64 (1.9.3-2) ...
Selecting previously unselected package libspatialindex-c6:amd64.
Preparing to unpack .../libspatialindex-c6_1.9.3-2_amd64.deb ...
Unpacking libspatialindex-c6:amd64 (1.9.3-2) ...
Selecting previously unselected package libspatialindex-dev:amd64.
Preparing to unpack .../libspatialindex-dev_1.9.3-2_amd64.deb ...
Unpacking libspatialindex-dev:amd64 (1.9.3-2) ...
Setting up libspatialindex6:amd64 (1.9.3-2) ...
Setting up libspatialindex-c6:amd64 (1.9.3-2) ...
Setting up libspatialindex-dev:amd64 (1.9.3-2) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local

In [ ]:
import os
import time
import json
import math
import requests
from datetime import datetime, timedelta, timezone

import numpy as np
import pandas as pd
from tqdm import tqdm

import osmnx as ox
import networkx as nx
import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely.ops import unary_union
import folium

ox.settings.log_console = False
ox.settings.use_cache = True

# ------------ CONFIG ------------
AREA_NAME = "Manhattan, New York, USA"
MODE = "walk"  # "walk" or "bike"

# Safety (NYC collisions)
CRASH_YEARS_BACK = 3
CRASH_MAX_RECORDS = 120000
CRASH_SEARCH_BUFFER_M = 20
SOCRATA_BASE = "https://data.cityofnewyork.us/resource/h9gi-nx95.json"
SOCRATA_VIEW_META = "https://data.cityofnewyork.us/api/views/h9gi-nx95.json"
SOCRATA_APP_TOKEN = os.getenv("NYC_SODA_APP_TOKEN", None)  # optional

# Greenery
GREEN_BUFFER_M = 25
GREEN_TAGS = {
    "leisure": ["park", "garden", "golf_course"],
    "landuse": ["recreation_ground", "meadow", "grass"],
    "natural": ["wood", "scrub", "heath"],
}

# Routing profiles
PROFILES = {
    "balanced": {"w_dist": 0.4, "w_risk": 0.4, "w_green": 0.2},
    "safer":    {"w_dist": 0.3, "w_risk": 0.6, "w_green": 0.1},
    "scenic":   {"w_dist": 0.3, "w_risk": 0.2, "w_green": 0.5},
}
SPEEDS_KMH = {"walk": 5.0, "bike": 14.0}

In [ ]:
def robust_min_max(x, q_low=0.05, q_high=0.95, clip=True):
    x = np.asarray(x, dtype=float)
    if not np.isfinite(x).any():
        return np.zeros_like(x, dtype=float)
    low = np.nanquantile(x, q_low)
    high = np.nanquantile(x, q_high)
    if high <= low:
        res = np.zeros_like(x, dtype=float)
    else:
        res = (x - low) / (high - low)
    return np.clip(res, 0, 1) if clip else res

# Replace the old geocode_point with this robust version
def geocode_point(s):
    import re
    s = str(s).strip()
    # Accept only numeric "lat,lon" (e.g., "40.758, -73.985")
    if re.match(r"^\s*-?\d+(\.\d+)?\s*,\s*-?\d+(\.\d+)?\s*$", s):
        a, b = [t.strip() for t in s.split(",", 1)]
        return float(a), float(b)  # lat, lon
    # Otherwise treat as address/place
    lat, lon = ox.geocode(s)  # returns (lat, lon)
    return lat, lon

def get_route_geometry(G, route):
    lines = []
    for u, v in zip(route[:-1], route[1:]):
        d = G.get_edge_data(u, v)
        if not d:
            d = G.get_edge_data(v, u)
            if not d:
                continue
        key = min(d, key=lambda k: d[k].get("length", 1))
        data = d[key]
        if "geometry" in data and data["geometry"] is not None:
            lines.append(data["geometry"])
        else:
            p1 = (G.nodes[u]["x"], G.nodes[u]["y"])
            p2 = (G.nodes[v]["x"], G.nodes[v]["y"])
            lines.append(LineString([p1, p2]))
    if not lines:
        return None
    if len(lines) == 1:
        return lines[0]
    try:
        from shapely.ops import linemerge
        return linemerge(lines)
    except Exception:
        coords = []
        for ln in lines:
            coords += list(ln.coords)
        return LineString(coords)

def get_socrata_columns(meta_url=SOCRATA_VIEW_META):
    try:
        r = requests.get(meta_url, timeout=60)
        r.raise_for_status()
        js = r.json()
        cols = [c.get("fieldName") for c in js.get("columns", []) if c.get("fieldName")]
        return set(cols)
    except Exception as e:
        print("Warning: could not fetch Socrata metadata:", e)
        return set()

def fetch_collisions_bbox(aoi_gdf, years_back=3, max_records=120000, app_token=None):
    # Build dynamic $select with columns that actually exist
    available = get_socrata_columns()
    base_fields = ["crash_date", "latitude", "longitude", "location"]
    injury_fields = []
    for cand in ["number_of_persons_injured", "persons_injured", "injured_persons"]:
        if cand in available:
            injury_fields.append(cand); break
    for cand in ["number_of_persons_killed", "persons_killed", "killed_persons"]:
        if cand in available:
            injury_fields.append(cand); break
    # Optional mode-specific (we'll include if present; otherwise we’ll ignore later)
    optional = []
    for cand in ["number_of_pedestrians_injured", "pedestrians_injured"]:
        if cand in available:
            optional.append(cand); break
    for cand in ["number_of_cyclists_injured", "number_of_cyclist_injured", "cyclists_injured"]:
        if cand in available:
            optional.append(cand); break

    select_fields = base_fields + injury_fields + optional
    select_fields = list(dict.fromkeys(select_fields))  # de-dup

    aoi_wgs = aoi_gdf.to_crs(4326)
    minx, miny, maxx, maxy = aoi_wgs.total_bounds
    since = (datetime.now(timezone.utc) - timedelta(days=365*years_back)).strftime("%Y-%m-%dT00:00:00.000")

    params_common = {
        "$select": ",".join(select_fields),
        "$where": f"crash_date >= '{since}' AND within_box(location, {maxy}, {minx}, {miny}, {maxx})",
        "$limit": 50000,
    }
    headers = {"X-App-Token": app_token} if app_token else {}
    frames, offset, total_fetched = [], 0, 0

    with tqdm(total=max_records, desc="Fetching NYC crashes", unit="rec") as pbar:
        while total_fetched < max_records:
            params = dict(params_common)
            params["$offset"] = offset
            r = requests.get(SOCRATA_BASE, params=params, headers=headers, timeout=60)
            if r.status_code != 200:
                print("Socrata error:", r.status_code, r.text[:200])
                break
            data = r.json()
            if not data:
                break
            df = pd.DataFrame(data)
            frames.append(df)
            n = len(df)
            total_fetched += n
            offset += n
            pbar.update(n)
            if n < params_common["$limit"]:
                break
            time.sleep(0.2)

    if not frames:
        return gpd.GeoDataFrame(columns=["geometry"], crs=4326)

    df = pd.concat(frames, ignore_index=True)

    # Coerce numerics if columns exist
    for col in ["latitude","longitude",
                "number_of_persons_injured","persons_injured","injured_persons",
                "number_of_persons_killed","persons_killed","killed_persons",
                "number_of_pedestrians_injured","pedestrians_injured",
                "number_of_cyclists_injured","number_of_cyclist_injured","cyclists_injured"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    if "crash_date" in df.columns:
        df["crash_date"] = pd.to_datetime(df["crash_date"], errors="coerce")

    df = df.dropna(subset=["latitude", "longitude"])
    gdf = gpd.GeoDataFrame(df,
                           geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
                           crs="EPSG:4326")
    # Clip to AOI polygon
    gdf = gdf[gdf.intersects(aoi_wgs.iloc[0].geometry)]
    return gdf

def series_from_first_existing(df, candidates, default_val=0.0):
    for c in candidates:
        if c in df.columns:
            return df[c].fillna(0)
    return pd.Series(default_val, index=df.index, dtype=float)

def compute_collision_risk_per_edge(edges_gdf, crashes_gdf, buffer_m=20, mode="walk"):
    # Helper to pick first existing column
    def _first_col(df, candidates, default=0.0):
        for c in candidates:
            if c in df.columns:
                return df[c].fillna(0)
        return pd.Series(default, index=df.index, dtype=float)

    if crashes_gdf.empty:
        edges_gdf["risk_raw"] = 0.0
        edges_gdf["risk_per_km"] = 0.0
        edges_gdf["risk_norm"] = 0.0
        return edges_gdf

    crashes_proj = crashes_gdf.to_crs(edges_gdf.crs).copy()

    # Dynamic severity weights (handle schema changes)
    killed = _first_col(crashes_proj, ["number_of_persons_killed","persons_killed","killed_persons"], 0.0)
    injured = _first_col(crashes_proj, ["number_of_persons_injured","persons_injured","injured_persons"], 0.0)
    ped_inj = _first_col(crashes_proj, ["number_of_pedestrians_injured","pedestrians_injured"], 0.0)
    cyc_inj = _first_col(crashes_proj, ["number_of_cyclists_injured","number_of_cyclist_injured","cyclists_injured"], 0.0)

    crashes_proj["sev_wt"] = killed*5.0 + injured*1.0 + (ped_inj if mode=="walk" else (cyc_inj if mode=="bike" else 0.0))

    # Nearest join; keep only matches within buffer_m (meters)
    join = gpd.sjoin_nearest(
        crashes_proj[["sev_wt", "geometry"]],
        edges_gdf[["geometry"]],
        how="left",
        distance_col="dist_to_edge",
        max_distance=buffer_m
    )

    # Use column access, not attribute; be robust to name variants
    idx_col = "index_right" if "index_right" in join.columns else next((c for c in join.columns if c.endswith("_right")), None)
    if idx_col is None or join.empty:
        edges_gdf["risk_raw"] = 0.0
        edges_gdf["risk_per_km"] = 0.0
        edges_gdf["risk_norm"] = 0.0
        return edges_gdf

    join = join.dropna(subset=[idx_col])
    if "dist_to_edge" in join.columns:
        join = join[join["dist_to_edge"] <= buffer_m]
    if join.empty:
        edges_gdf["risk_raw"] = 0.0
        edges_gdf["risk_per_km"] = 0.0
        edges_gdf["risk_norm"] = 0.0
        return edges_gdf

    # Aggregate crash severity to edges by the right index
    sev_per_edge = join.groupby(join[idx_col])["sev_wt"].sum()

    # Initialize and assign
    edges_gdf["risk_raw"] = 0.0
    # sev_per_edge.index holds the edge indices (works with MultiIndex too)
    edges_gdf.loc[sev_per_edge.index, "risk_raw"] = sev_per_edge.values

    # Per-km with smoothing + normalization
    length_km = edges_gdf["length_m"] / 1000.0
    edges_gdf["risk_per_km"] = edges_gdf["risk_raw"] / (length_km + 0.02)
    edges_gdf["risk_norm"] = robust_min_max(edges_gdf["risk_per_km"].values)
    return edges_gdf

def fetch_osm_green_polygons(area_name, tags):
    gdf = ox.geometries_from_place(area_name, tags)
    if gdf.empty:
        return gpd.GeoDataFrame(columns=["geometry"], crs="EPSG:4326")
    gdf = gdf[gdf.geometry.notna()]
    gdf = gdf[gdf.geometry.geom_type.isin(["Polygon", "MultiPolygon"])]
    return gdf.to_crs(4326)

def compute_green_overlap_fraction(edges_proj, greens_proj, buffer_m=25):
    if greens_proj.empty:
        edges_proj["green_frac"] = 0.0
        edges_proj["green_norm"] = 0.0
        return edges_proj

    greens_proj = greens_proj.to_crs(edges_proj.crs).buffer(0)
    try:
        green_union = unary_union(list(greens_proj.geometry))
    except Exception:
        green_union = unary_union(greens_proj.geometry)

    def frac_for_geom(geom):
        try:
            buf = geom.buffer(buffer_m)
            inter_area = buf.intersection(green_union).area
            return float(inter_area / buf.area) if buf.area > 0 else 0.0
        except Exception:
            return 0.0

    values = []
    CHUNK = 4000
    geoms = list(edges_proj.geometry.values)
    for i in tqdm(range(0, len(geoms), CHUNK), desc="Computing greenery per edge"):
        chunk = geoms[i:i+CHUNK]
        vals = [frac_for_geom(g) for g in chunk]
        values.extend(vals)

    edges_proj["green_frac"] = np.array(values, dtype=float)
    edges_proj["green_norm"] = robust_min_max(edges_proj["green_frac"].values)
    return edges_proj

def attach_edge_attribute(G, edges_gdf, colname, new_attr=None):
    name = new_attr or colname
    mapping = edges_gdf[colname].to_dict()  # keys are (u,v,key)
    nx.set_edge_attributes(G, mapping, name)

def compute_cost(edges_gdf, w_dist=0.4, w_risk=0.4, w_green=0.2):
    edges_gdf["dist_norm"] = robust_min_max(edges_gdf["length_m"].values)
    if "risk_norm" not in edges_gdf.columns:
        edges_gdf["risk_norm"] = 0.0
    if "green_norm" not in edges_gdf.columns:
        edges_gdf["green_norm"] = 0.0
    edges_gdf["cost_total"] = (
        w_dist * edges_gdf["dist_norm"] +
        w_risk * edges_gdf["risk_norm"] +
        w_green * (1.0 - edges_gdf["green_norm"])
    )
    return edges_gdf

def fetch_elevations_opentopodata(lats, lons, provider="srtm30m"):
    base = f"https://api.opentopodata.org/v1/{provider}"
    coords = [f"{lat},{lon}" for lat, lon in zip(lats, lons)]
    elevations = []
    BATCH = 100
    for i in range(0, len(coords), BATCH):
        chunk = coords[i:i+BATCH]
        url = base + "?locations=" + "|".join(chunk)
        try:
            r = requests.get(url, timeout=60)
            if r.status_code != 200:
                elevations.extend([0.0]*len(chunk))
                continue
            js = r.json()
            results = js.get("results", [])
            for res in results:
                elevations.append(res.get("elevation", 0.0))
            time.sleep(0.1)
        except Exception:
            elevations.extend([0.0]*len(chunk))
    return elevations

def compute_elevation_gain_for_route(G, route):
    if not route:
        return 0.0
    lats = [G.nodes[n]["y"] for n in route]
    lons = [G.nodes[n]["x"] for n in route]
    elevs = fetch_elevations_opentopodata(lats, lons)
    gain = 0.0
    for a, b in zip(elevs[:-1], elevs[1:]):
        dz = (b or 0.0) - (a or 0.0)
        if dz > 0:
            gain += dz
    return float(gain)

def compute_route_metrics(G, edges_gdf, route, mode="walk"):
    if not route:
        return dict(distance_km=0.0, eta_min=0.0, risk_score=0.0, green_score=0.0, elev_gain_m=0.0)

    dist_m = 0.0
    risks, greens, lengths_km = [], [], []

    for u, v in zip(route[:-1], route[1:]):
        d = G.get_edge_data(u, v)
        if not d:
            d = G.get_edge_data(v, u)
            if not d:
                continue
        k = min(d, key=lambda kk: d[kk].get("length", 1))
        length = d[k].get("length", 0.0)
        dist_m += length

        idx = (u, v, k) if (u, v, k) in edges_gdf.index else ((v, u, k) if (v, u, k) in edges_gdf.index else None)
        if idx is not None:
            row = edges_gdf.loc[idx]
            risks.append(row.get("risk_norm", 0.0))
            greens.append(row.get("green_norm", 0.0))
            lengths_km.append(row.get("length_m", 0.0) / 1000.0)

    w = np.array(lengths_km) if lengths_km else np.array([1.0])
    risk_score = float(np.average(np.array(risks) if risks else np.array([0.0]), weights=w))
    green_score = float(np.average(np.array(greens) if greens else np.array([0.0]), weights=w))

    speed_kmh = SPEEDS_KMH.get(mode, 5.0)
    eta_min = (dist_m / 1000.0) / speed_kmh * 60.0
    elev_gain_m = compute_elevation_gain_for_route(G, route)
    return {"distance_km": dist_m/1000.0, "eta_min": eta_min, "risk_score": risk_score, "green_score": green_score, "elev_gain_m": elev_gain_m}

In [ ]:
print("Loading boundary and street network...")
aoi = ox.geocode_to_gdf(AREA_NAME)
G = ox.graph_from_place(AREA_NAME, network_type=MODE, simplify=True)

# Keep only the largest weakly connected component (fix for earlier error)
largest = max(nx.weakly_connected_components(G), key=len)
G = G.subgraph(largest).copy()

nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True, fill_edge_geometry=True)

# Ensure length in meters
edges["length_m"] = edges["length"] if "length" in edges.columns else edges.geometry.length

# Project to metric CRS
edges_proj = edges.to_crs(aoi.estimate_utm_crs())
print(f"Edges: {len(edges_proj):,}, CRS: {edges_proj.crs}")

Loading boundary and street network...
Edges: 163,540, CRS: EPSG:32618


In [ ]:
print("Fetching OSM greenery polygons...")
greens = fetch_osm_green_polygons(AREA_NAME, GREEN_TAGS)
if greens.empty:
    print("No green polygons found. Green score = 0.")
    edges_proj["green_norm"] = 0.0
else:
    greens_proj = greens.to_crs(edges_proj.crs)
    edges_proj = compute_green_overlap_fraction(edges_proj, greens_proj, buffer_m=GREEN_BUFFER_M)
    print("Green features computed.")

Fetching OSM greenery polygons...


/tmp/ipython-input-1011596745.py:216: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  gdf = ox.geometries_from_place(area_name, tags)
Computing greenery per edge:   0%|          | 0/41 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
Computing greenery per edge: 100%|██████████| 41/41 [05:52<00:00,  8.60s/it]

Green features computed.



/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
print("Fetching NYC collisions and computing risk...")
crashes = fetch_collisions_bbox(aoi,
                                years_back=CRASH_YEARS_BACK,
                                max_records=CRASH_MAX_RECORDS,
                                app_token=SOCRATA_APP_TOKEN)
if crashes.empty:
    print("No crashes fetched; risk will be zero.")
else:
    print(f"Crashes fetched: {len(crashes):,}")

edges_proj = compute_collision_risk_per_edge(edges_proj, crashes,
                                             buffer_m=CRASH_SEARCH_BUFFER_M,
                                             mode=MODE)

Fetching NYC collisions and computing risk...


Fetching NYC crashes:   0%|          | 0/120000 [00:00<?, ?rec/s]/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
Fetching NYC crashes:  73%|███████▎  | 87416/120000 [00:10<00:03, 8514.28rec/s]


Crashes fetched: 44,539


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
edges_combined = edges_proj.copy()

def apply_profile_and_route(G, edges_df, source_node, target_node, profile):
    e = edges_df.copy()
    e = compute_cost(e, w_dist=profile["w_dist"], w_risk=profile["w_risk"], w_green=profile["w_green"])
    attach_edge_attribute(G, e, "cost_total", new_attr="cost_total")
    route = nx.shortest_path(G, source=source_node, target=target_node, weight="cost_total")
    return route, e

def compute_baseline_route(G, source_node, target_node):
    attach_edge_attribute(G, edges_combined, "length_m", new_attr="length_m")
    return nx.shortest_path(G, source=source_node, target=target_node, weight="length_m")

# Origin/Destination
ORIGIN = "Times Square, Manhattan, New York, USA"
DEST   = "Brooklyn Bridge, Manhattan, New York, USA"

orig_lat, orig_lon = geocode_point(ORIGIN)
dest_lat, dest_lon = geocode_point(DEST)
orig_node = ox.nearest_nodes(G, X=orig_lon, Y=orig_lat)
dest_node = ox.nearest_nodes(G, X=dest_lon, Y=dest_lat)

print("Computing routes...")
routes = {"shortest": compute_baseline_route(G, orig_node, dest_node)}
for name, prof in PROFILES.items():
    r, _ = apply_profile_and_route(G, edges_combined, orig_node, dest_node, prof)
    routes[name] = r

metrics = {name: compute_route_metrics(G, edges_combined, r, mode=MODE) for name, r in routes.items()}
pd.DataFrame(metrics).T.round({"distance_km": 3, "eta_min": 1, "risk_score": 3, "green_score": 3, "elev_gain_m": 1})

Computing routes...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,distance_km,eta_min,risk_score,green_score,elev_gain_m
shortest,6.315,75.8,0.0,0.078,269.0
balanced,8.638,103.7,0.0,0.189,150.0
safer,8.549,102.6,0.0,0.154,64.0
scenic,8.726,104.7,0.0,0.223,0.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
center = [orig_lat, orig_lon]
m = folium.Map(location=center, tiles="CartoDB positron", zoom_start=13)

colors = {"shortest": "#666666", "balanced": "#2F80ED", "safer": "#F2994A", "scenic": "#27AE60"}
styles = {"shortest": {"weight": 4, "opacity": 0.6, "dash_array": "8,8"},
          "balanced": {"weight": 5, "opacity": 0.9},
          "safer":    {"weight": 5, "opacity": 0.9},
          "scenic":   {"weight": 5, "opacity": 0.9}}

folium.Marker([orig_lat, orig_lon], popup="Origin", icon=folium.Icon(color="blue", icon="play")).add_to(m)
folium.Marker([dest_lat, dest_lon], popup="Destination", icon=folium.Icon(color="red", icon="flag")).add_to(m)

def add_route_layer(G, route, name, color, style):
    geom = get_route_geometry(G, route)
    if geom is None:
        return
    coords = [(lat, lon) for lon, lat in geom.coords]
    pop = metrics[name]
    popup_html = f"<b>{name.title()} route</b><br>" \
                 f"Distance: {pop['distance_km']:.2f} km<br>" \
                 f"ETA: {pop['eta_min']:.0f} min<br>" \
                 f"Risk score: {pop['risk_score']:.2f}<br>" \
                 f"Green score: {pop['green_score']:.2f}<br>" \
                 f"Elev. gain: {pop['elev_gain_m']:.0f} m"
    folium.PolyLine(locations=coords, color=color, popup=popup_html, **style).add_to(m)

for name in ["shortest", "balanced", "safer", "scenic"]:
    add_route_layer(G, routes[name], name, colors[name], styles[name])

m

In [ ]:
df = pd.DataFrame(metrics).T
baseline = df.loc["shortest"].copy()
comp = df.copy()
for col in ["distance_km", "eta_min", "risk_score", "green_score", "elev_gain_m"]:
    comp[f"{col}_vs_shortest_%"] = (comp[col] - baseline[col]) / max(baseline[col], 1e-9) * 100.0

display(df.round(3))
display(comp[[c for c in comp.columns if c.endswith("_vs_shortest_%")]].round(1))

,distance_km,eta_min,risk_score,green_score,elev_gain_m
shortest,6.315,75.785,0.0,0.078,269.0
balanced,8.638,103.658,0.0,0.189,150.0
safer,8.549,102.591,0.0,0.154,64.0
scenic,8.726,104.711,0.0,0.223,0.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,distance_km_vs_shortest_%,eta_min_vs_shortest_%,risk_score_vs_shortest_%,green_score_vs_shortest_%,elev_gain_m_vs_shortest_%
shortest,0.0,0.0,0.0,0.0,0.0
balanced,36.8,36.8,0.0,140.4,-44.2
safer,35.4,35.4,0.0,96.3,-76.2
scenic,38.2,38.2,0.0,184.1,-100.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
ox.save_graphml(G, "city_graph.graphml")
edges_out = edges_combined[["length_m", "risk_norm", "green_norm"]].reset_index()  # keep u,v,key
edges_out.to_parquet("edges_features.parquet", index=False)

aoi_center = aoi.to_crs(4326).geometry.iloc[0].centroid
with open("aoi_meta.json", "w") as f:
    json.dump({"center_lat": aoi_center.y, "center_lon": aoi_center.x, "area_name": AREA_NAME, "mode": MODE}, f)

print("Saved: city_graph.graphml, edges_features.parquet, aoi_meta.json")

In [ ]:
%%writefile app.py
import json, re
import numpy as np
import pandas as pd
import networkx as nx
import osmnx as ox
from shapely.geometry import LineString, MultiLineString
import folium
from streamlit_folium import st_folium
import streamlit as st

st.set_page_config(page_title="Safe & Scenic Route Finder", layout="wide")

@st.cache_resource
def load_graph(path="city_graph.graphml"):
    return ox.load_graphml(path)

@st.cache_data
def load_edges_features(path="edges_features.parquet"):
    df = pd.read_parquet(path)
    # Enforce index dtypes to match NetworkX edge keys
    for c in ("u","v","key"):
        if c not in df.columns:
            raise ValueError(f"Missing column in edges_features: {c}")
    df["u"] = pd.to_numeric(df["u"], errors="coerce").fillna(0).astype(np.int64)
    df["v"] = pd.to_numeric(df["v"], errors="coerce").fillna(0).astype(np.int64)
    df["key"] = pd.to_numeric(df["key"], errors="coerce").fillna(0).astype(np.int64)
    df.set_index(["u","v","key"], inplace=True)
    # Safety: ensure required feature cols exist
    for col in ["length_m","risk_norm","green_norm"]:
        if col not in df.columns:
            df[col] = 0.0
    return df

@st.cache_data
def load_meta(path="aoi_meta.json"):
    with open(path, "r") as f:
        return json.load(f)

def robust_min_max(x):
    x = np.asarray(x, dtype=float)
    if not np.isfinite(x).any(): return np.zeros_like(x)
    low, high = np.nanquantile(x, 0.05), np.nanquantile(x, 0.95)
    if high <= low: return np.zeros_like(x)
    return np.clip((x - low) / (high - low), 0, 1)

def apply_cost(G, edges_df, w_dist, w_risk, w_green):
    e = edges_df.copy()
    if "dist_norm" not in e.columns:
        e["dist_norm"] = robust_min_max(e["length_m"].values)
    if "risk_norm" not in e.columns:
        e["risk_norm"] = 0.0
    if "green_norm" not in e.columns:
        e["green_norm"] = 0.0
    e["cost_total"] = (
        w_dist * e["dist_norm"] +
        w_risk * e["risk_norm"] +
        w_green * (1.0 - e["green_norm"])
    )
    nx.set_edge_attributes(G, e["cost_total"].to_dict(), "cost_total")

def geocode_point(s):
    s = str(s).strip()
    # Only treat as lat,lon if both tokens are numeric
    if re.match(r"^\s*-?\d+(\.\d+)?\s*,\s*-?\d+(\.\d+)?\s*$", s):
        a, b = [t.strip() for t in s.split(",", 1)]
        return float(a), float(b)
    lat, lon = ox.geocode(s)  # returns (lat, lon)
    return lat, lon

def coords_from_geom(geom):
    # Return list of [lat, lon] for LineString or MultiLineString
    def _line_to_latlon(g):
        return [[float(y), float(x)] for x, y in g.coords]
    if isinstance(geom, LineString):
        return _line_to_latlon(geom)
    if isinstance(geom, MultiLineString):
        pts = []
        for g in geom.geoms:
            pts.extend(_line_to_latlon(g))
        return pts
    return []

def route_to_coords(G, route):
    coords = []
    for u, v in zip(route[:-1], route[1:]):
        d = G.get_edge_data(u, v)
        if not d: d = G.get_edge_data(v, u)
        if not d: continue
        k = min(d, key=lambda kk: d[kk].get("length", 1))
        geom = d[k].get("geometry", None)
        if geom is None:
            p1 = [G.nodes[u]["y"], G.nodes[u]["x"]]
            p2 = [G.nodes[v]["y"], G.nodes[v]["x"]]
            coords += [p1, p2]
        else:
            coords += coords_from_geom(geom)
    return coords

def route_metrics(G, edges_df, route, speed_kmh):
    dist_m = 0.0
    risks, greens, lengths_km = [], [], []
    for u, v in zip(route[:-1], route[1:]):
        d = G.get_edge_data(u, v)
        if not d: d = G.get_edge_data(v, u)
        if not d: continue
        k = min(d, key=lambda kk: d[kk].get("length", 1))
        length = d[k].get("length", 0.0)
        dist_m += length
        idx = (u, v, k) if (u, v, k) in edges_df.index else ((v, u, k) if (v, u, k) in edges_df.index else None)
        if idx is not None:
            row = edges_df.loc[idx]
            risks.append(row.get("risk_norm", 0.0))
            greens.append(row.get("green_norm", 0.0))
            lengths_km.append(row.get("length_m", 0.0)/1000.0)

    w = np.array(lengths_km) if lengths_km else np.array([1.0])
    risk = float(np.average(np.array(risks) if risks else np.array([0.0]), weights=w))
    green = float(np.average(np.array(greens) if greens else np.array([0.0]), weights=w))
    eta_min = (dist_m/1000.0)/speed_kmh*60.0 if speed_kmh>0 else 0.0
    return dist_m/1000.0, eta_min, risk, green

# Load data
meta = load_meta()
G = load_graph()
edges_df = load_edges_features()

st.title("Safe & Scenic Route Finder")

# Sidebar/cache tools
with st.sidebar:
    if st.button("Clear Streamlit cache"):
        st.cache_data.clear(); st.cache_resource.clear()
        st.success("Cache cleared. Reload the page.")

left, right = st.columns([1,2])

with left:
    st.subheader("Inputs")
    default_o = "Times Square, Manhattan, New York, USA"
    default_d = "Brooklyn Bridge, Manhattan, New York, USA"
    origin = st.text_input("Origin (address or 'lat,lon')", default_o)
    dest = st.text_input("Destination (address or 'lat,lon')", default_d)
    mode = st.selectbox("Mode", ["walk", "bike"], index=0)
    speed_kmh = 5.0 if mode=="walk" else 14.0

    st.markdown("Weights")
    w_dist = st.slider("Distance weight", 0.0, 1.0, 0.4, 0.05)
    w_risk = st.slider("Safety weight",   0.0, 1.0, 0.4, 0.05)
    w_green= st.slider("Scenic weight",   0.0, 1.0, 0.2, 0.05)

    go = st.button("Compute Route")

with right:
    st.subheader("Map")
    center = [meta["center_lat"], meta["center_lon"]]
    fmap = folium.Map(location=center, tiles="CartoDB positron", zoom_start=13)

    if go:
        try:
            olat, olon = geocode_point(origin)
            dlat, dlon = geocode_point(dest)
            onode = ox.nearest_nodes(G, X=olon, Y=olat)
            dnode = ox.nearest_nodes(G, X=dlon, Y=dlat)

            # Baseline shortest by length_m (if any edge lacks it, use length fallback = 1)
            nx.set_edge_attributes(G, edges_df["length_m"].to_dict(), "length_m")
            shortest = nx.shortest_path(G, onode, dnode, weight="length_m")

            # Weighted route
            apply_cost(G, edges_df, w_dist, w_risk, w_green)
            weighted = nx.shortest_path(G, onode, dnode, weight="cost_total")

            # Convert to polylines and fit map
            s_coords = route_to_coords(G, shortest)
            w_coords = route_to_coords(G, weighted)

            # Markers
            folium.Marker([olat, olon], popup="Origin", icon=folium.Icon(color="blue", icon="play")).add_to(fmap)
            folium.Marker([dlat, dlon], popup="Destination", icon=folium.Icon(color="red", icon="flag")).add_to(fmap)

            # Routes
            if s_coords:
                folium.PolyLine(s_coords, color="#666666", weight=4, opacity=0.7, dash_array="8,8",
                                popup="Shortest").add_to(fmap)
            if w_coords:
                folium.PolyLine(w_coords, color="#2F80ED", weight=5, opacity=0.95,
                                popup="Weighted").add_to(fmap)

            # Fit bounds to show routes
            all_coords = s_coords + w_coords
            if all_coords:
                lats = [c[0] for c in all_coords]; lons = [c[1] for c in all_coords]
                sw = [min(lats), min(lons)]
                ne = [max(lats), max(lons)]
                fmap.fit_bounds([sw, ne])

            # Metrics table
            s_km, s_eta, s_r, s_g = route_metrics(G, edges_df, shortest, speed_kmh)
            w_km, w_eta, w_r, w_g = route_metrics(G, edges_df, weighted, speed_kmh)
            st_folium(fmap, width=900, height=600)
            st.subheader("Metrics")
            df = pd.DataFrame({
                "route": ["Shortest", "Weighted"],
                "distance_km": [s_km, w_km],
                "eta_min": [s_eta, w_eta],
                "risk_score": [s_r, w_r],
                "green_score": [s_g, w_g],
            }).round(3)
            st.dataframe(df, use_container_width=True)

        except nx.NetworkXNoPath:
            st.error("No path found between the selected points in this network (try a closer destination or change mode).")
        except Exception as e:
            st.exception(e)
    else:
        st_folium(fmap, width=900, height=600)

st.caption("Data: OSM (ODbL), NYC Open Data (collisions, precomputed), experimental routing — not for navigation.")

Writing app.py


In [ ]:
# Kill previous Streamlit (if any)
!pkill -f "streamlit run app.py" || true

from pyngrok import ngrok
import time

NGROK_TOKEN = "33vYIxPhgkHiBvZ9O5FpTmv84Le_7eka6RpEvy4vYtVHWMj4"  # replace with your token
if NGROK_TOKEN and "PASTE_YOUR_TOKEN_HERE" not in NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)

tunnel = ngrok.connect(8501)
print("Public URL:", tunnel.public_url)

!streamlit run app.py --server.port 8501 --server.headless true &> streamlit.log &
time.sleep(4)
!tail -n 80 streamlit.log